In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [2]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [3]:
def clean(df):
    df['income']=df['ApplicantIncome']+df['CoapplicantIncome']
    df['loanpm']=df['LoanAmount']+df['Loan_Amount_Term']
    df.drop(['LoanAmount','Loan_ID','Loan_Amount_Term','ApplicantIncome','CoapplicantIncome'],axis='columns',inplace=True)
    df['Gender']=df['Gender'].replace({'Male':1,'Female':0})
    df['Married']=df['Married'].replace({'Yes':1,'No':0})
    df['Education']=df['Education'].replace({'Graduate':1,'Not Graduate':0})
    df['Self_Employed']=df['Self_Employed'].replace({'Yes':1,'No':0})
    df['Dependents']=df['Dependents'].replace({'3+':3,'1':1,'2':2})
    df['Property_Area']=df['Property_Area'].replace({'Urban':3,'Rural':1,'Semiurban':2})
    df['income_bins']=pd.cut(df.income,np.linspace(1400,81000,dtype='int',num=20),labels=np.linspace(0,18,dtype='int',num=19))
    df['Income']=pd.cut(df.income_bins,[-1,0,1,2,3,18],labels=[0,1,2,3,4])
    df['loan']=pd.cut(df.loanpm,np.linspace(100,1000,dtype='int',num=10),labels=[0,1,2,3,4,5,6,7,8])
    df.drop(['income','loanpm','income_bins'],axis=1,inplace=True)
    df=df.dropna()
    df.Dependents=df.Dependents.astype('int')
    return df

In [4]:
def get_filled(test1,col):
    test1=test.loc[test[col].isnull()]
    test1[col]='0'
    print(test1.isnull().sum())
    return test1

In [5]:
def new_df(df1,df2,col,df3):
    df_new=pd.concat([df1,df2],axis=0)
    df_new.drop('Loan_Status',axis=1,inplace=True)
    df_new=clean(df_new)
    x=df_new.drop([col],axis=1)
    y=df_new[col]
    lreg=LogisticRegression()
    lreg.fit(x,y)
    df3[col]=lreg.predict(df3.drop([col],axis=1))
    a=df2.loc[df2[col].notnull(),col]
    b=df3[col]
    c=pd.concat([a,b],axis=0)
    c.sort_index(axis=0,ascending=True,inplace=True)
    df2.drop(col,axis=1,inplace=True)
    df2=pd.concat([df2,c],axis=1)
    return df2
    #df1==df(cleaned)
    #df2==test(cleaned wrt col)
    #df3==test1(cleaned)

In [6]:
df=pd.read_csv('train.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
test0=pd.read_csv('test.csv')
test=test0.copy()
test0.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [8]:
test.isnull().sum()

Loan_ID               0
Gender               11
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
dtype: int64

In [9]:
test1=get_filled(test,'Gender')

Loan_ID              0
Gender               0
Married              0
Dependents           1
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64


In [10]:
test1.Dependents.replace({np.nan:0},inplace=True)
test1=clean(test1)
test1.isnull().sum()

Gender            0
Married           0
Dependents        0
Education         0
Self_Employed     0
Credit_History    0
Property_Area     0
Income            0
loan              0
dtype: int64

In [11]:
test['Gender']=test['Gender'].replace({'Male':1,'Female':0})
test=new_df(df,test,'Gender',test1)
test.isnull().sum()

Loan_ID               0
Married               0
Dependents           10
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
Gender                0
dtype: int64

# gender done

In [12]:
test1=get_filled(test,'Dependents')

Loan_ID              0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           1
Loan_Amount_Term     0
Credit_History       2
Property_Area        0
Gender               0
dtype: int64


In [13]:
test1.Credit_History.mode()

0    1.0
dtype: float64

In [14]:
test1.LoanAmount.median()

104.0

In [15]:
test1.Credit_History.replace({np.nan:1},inplace=True)
test1.LoanAmount.replace({np.nan:104},inplace=True)
test1['Gender']=test1['Gender'].replace({1:'Male',0:'Female'})

In [16]:
test1=clean(test1)
test1.isnull().sum()

Married           0
Dependents        0
Education         0
Self_Employed     0
Credit_History    0
Property_Area     0
Gender            0
Income            0
loan              0
dtype: int64

In [17]:
test['Dependents']=test['Dependents'].replace({'3+':3,'1':1,'2':2})
test=new_df(df,test,'Dependents',test1)
test.isnull().sum()

Loan_ID               0
Married               0
Education             0
Self_Employed        23
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
Gender                0
Dependents            0
dtype: int64

# dependents done

In [18]:
test1=get_filled(test,'Self_Employed')

Loan_ID              0
Married              0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       1
Property_Area        0
Gender               0
Dependents           0
dtype: int64


In [19]:
test1.Credit_History.replace({np.nan:1},inplace=True)
test1['Gender']=test1['Gender'].replace({1:'Male',0:'Female'})
test1['Dependents']=test1['Dependents'].replace({'3+':3,'1':1,'2':2})
test1=clean(test1)
test1.isnull().sum()

Married           0
Education         0
Self_Employed     0
Credit_History    0
Property_Area     0
Gender            0
Dependents        0
Income            0
loan              0
dtype: int64

In [20]:
test['Self_Employed']=test['Self_Employed'].replace({'Yes':1,'No':0})
test=new_df(df,test,'Self_Employed',test1)
test.isnull().sum()

Loan_ID               0
Married               0
Education             0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            5
Loan_Amount_Term      6
Credit_History       29
Property_Area         0
Gender                0
Dependents            0
Self_Employed         0
dtype: int64

# self employed done

In [21]:
test1=get_filled(test,'Credit_History')

Loan_ID              0
Married              0
Education            0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     1
Credit_History       0
Property_Area        0
Gender               0
Dependents           0
Self_Employed        0
dtype: int64


In [22]:
test1.Loan_Amount_Term.median()

360.0

In [23]:
test1.Loan_Amount_Term.replace({np.nan:360},inplace=True)
test1['Gender']=test1['Gender'].replace({1:'Male',0:'Female'})
test1['Dependents']=test1['Dependents'].replace({'3+':3,'1':1,'2':2})
test1['Self_Employed']=test1['Self_Employed'].replace({1:'Yes',0:'No'})
test1=clean(test1)
test1.isnull().sum()

Married           0
Education         0
Credit_History    0
Property_Area     0
Gender            0
Dependents        0
Self_Employed     0
Income            0
loan              0
dtype: int64

In [24]:
test=new_df(df,test,'Credit_History',test1)
test.isnull().sum()

Loan_ID              0
Married              0
Education            0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           5
Loan_Amount_Term     6
Property_Area        0
Gender               0
Dependents           0
Self_Employed        0
Credit_History       0
dtype: int64

# credit history done

In [25]:
test.LoanAmount.median()

125.0

In [26]:
test.Loan_Amount_Term.median()

360.0

In [27]:
test.LoanAmount.replace({np.nan:125},inplace=True)
test.Loan_Amount_Term.replace({np.nan:360},inplace=True)
test.isnull().sum()

Loan_ID              0
Married              0
Education            0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Property_Area        0
Gender               0
Dependents           0
Self_Employed        0
Credit_History       0
dtype: int64

In [28]:
test.head()

,Loan_ID,Married,Education,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Property_Area,Gender,Dependents,Self_Employed,Credit_History
0,LP001015,Yes,Graduate,5720,0,110.0,360.0,Urban,1.0,0,0.0,1.0
1,LP001022,Yes,Graduate,3076,1500,126.0,360.0,Urban,1.0,1,0.0,1.0
2,LP001031,Yes,Graduate,5000,1800,208.0,360.0,Urban,1.0,2,0.0,1.0
3,LP001035,Yes,Graduate,2340,2546,100.0,360.0,Urban,1.0,2,0.0,1.0
4,LP001051,No,Not Graduate,3276,0,78.0,360.0,Urban,1.0,0,0.0,1.0


In [29]:
test['Gender']=test['Gender'].replace({1:'Male',0:'Female'})
test['Dependents']=test['Dependents'].replace({3:'3+',1:'1',2:'2'})
test['Self_Employed']=test['Self_Employed'].replace({1:'Yes',0:'No'})
test=clean(test)

In [30]:
test.head()

,Married,Education,Property_Area,Gender,Dependents,Self_Employed,Credit_History,Income,loan
0,1,1,3,1,0,0,1.0,1,3
1,1,1,3,1,1,0,1.0,0,3
2,1,1,3,1,2,0,1.0,1,4
3,1,1,3,1,2,0,1.0,0,3
4,0,0,3,1,0,0,1.0,0,3


In [31]:
test=test[['Gender','Married','Dependents','Education','Self_Employed','Credit_History','Property_Area','Income','loan']]
test.head()

,Gender,Married,Dependents,Education,Self_Employed,Credit_History,Property_Area,Income,loan
0,1,1,0,1,0,1.0,3,1,3
1,1,1,1,1,0,1.0,3,0,3
2,1,1,2,1,0,1.0,3,1,4
3,1,1,2,1,0,1.0,3,0,3
4,1,0,0,0,0,1.0,3,0,3


In [32]:
test.to_csv('test_modified.csv',index=False)